<a href="https://colab.research.google.com/github/shahin1009/EDMD/blob/main/EDMD.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np

def buildTheta(yin, nVars, polyorder):

    n = yin.shape[0]
    ind = 0
    yout = np.zeros((n, nVars))

    # Copy input variables to output
    for i in np.arange(nVars):
        yout[:, ind] = yin[:, i]
        ind += 1

    if polyorder >= 2:
        # Polynomial order 2
        for i in np.arange(nVars):
            for j in np.arange(i, nVars):
                new_col = yin[:, i] * yin[:, j]
                yout = np.append(yout, new_col.reshape(-1, 1), 1)
                ind += 1

    if polyorder >= 3:
        # Polynomial order 3
        for i in np.arange(nVars):
            for j in np.arange(i, nVars):
                for k in np.arange(j, nVars):
                    new_col = yin[:, i] * yin[:, j] * yin[:, k]
                    yout = np.append(yout, new_col.reshape(-1, 1), 1)
                    ind += 1

    if polyorder >= 4:
        # Polynomial order 4
        for i in np.arange(nVars):
            for j in np.arange(i, nVars):
                for k in np.arange(j, nVars):
                    for l in np.arange(k, nVars):
                        new_col = yin[:, i] * yin[:, j] * yin[:, k] * yin[:, l]
                        yout = np.append(yout, new_col.reshape(-1, 1), 1)
                        ind += 1


    return yout


In [ ]:
def buildGamma(yin, ydotin, nVars, polyorder):
    n = yin.shape[0]
    ind = 0
    yout = np.zeros((n, nVars))
    for i in np.arange(nVars):
        yout[:, ind] = ydotin[:, i]
        ind += 1

    if polyorder >= 2:
        # poly order 2
        for i in np.arange(nVars):
            for j in np.arange(i, nVars):
                new_col = ydotin[:, i] * yin[:, j] + yin[:, i] * ydotin[:, j]
                yout = np.append(yout, new_col.reshape(-1, 1), 1)
                ind += 1

    if polyorder >= 3:
        # poly order 3
        for i in np.arange(nVars):
            for j in np.arange(i, nVars):
                for k in np.arange(j, nVars):
                    new_col = (
                        ydotin[:, i] * yin[:, j] * yin[:, k]
                        + yin[:, i] * ydotin[:, j] * yin[:, k]
                        + yin[:, i] * yin[:, j] * ydotin[:, k]
                    )
                    yout = np.append(yout, new_col.reshape(-1, 1), 1)
                    ind += 1

    if polyorder >= 4:
        # poly order 4
        for i in np.arange(nVars):
            for j in np.arange(i, nVars):
                for k in np.arange(j, nVars):
                    for l in np.arange(k, nVars):
                        new_col = (
                            ydotin[:, i] * yin[:, j] * yin[:, k] * yin[:, l]
                            + yin[:, i] * ydotin[:, j] * yin[:, k] * yin[:, l]
                            + yin[:, i] * yin[:, j] * ydotin[:, k] * yin[:, l]
                            + yin[:, i] * yin[:, j] * yin[:, k] * ydotin[:, l]
                        )
                        yout = np.append(yout, new_col.reshape(-1, 1), 1)
                        ind += 1

    return yout


In [ ]:
import numpy as np

def poolDataLIST(yin, ahat, nVars, polyorder):
    n = len(yin)+1
    ind = 0
    yout = []

    for i in range(nVars):
        yout.append([yin[i]])
        ind += 1

    if polyorder >= 2:
        for i in range(nVars):
            for j in range(i, nVars):
                yout.append([yin[i], yin[j]])
                ind += 1

    if polyorder >= 3:
        for i in range(nVars):
            for j in range(i, nVars):
                for k in range(j, nVars):
                    yout.append([yin[i], yin[j], yin[k]])
                    ind += 1

    if polyorder >= 4:
        for i in range(nVars):
            for j in range(i, nVars):
                for k in range(j, nVars):
                    for l in range(k, nVars):
                        yout.append([yin[i], yin[j], yin[k], yin[l]])
                        ind += 1

    if polyorder >= 5:
        for i in range(nVars):
            for j in range(i, nVars):
                for k in range(j, nVars):
                    for l in range(k, nVars):
                        for m in range(l, nVars):
                            yout.append([yin[i], yin[j], yin[k], yin[l], yin[m]])
                            ind += 1

    output = yout

    for item1, item2 in zip(output, ahat):
      print(f'{item1}\t{item2}')
    return output


In [ ]:
from scipy.integrate import odeint
from scipy.linalg import svd

def f(q, t):
    x, y = q
    return [y, x-x**3]

def solve(ic,t):
    sol = odeint(f, ic, t)
    return sol

t = np.linspace(0, 10, 10000)
x0 = np.array([0, -2.8])
y=solve(x0,t)
dy = np.array(f(y.T,t)).T

In [ ]:

# Construct libraries
polyorder = 4
nvar = 2
Theta = buildTheta(y, nvar, polyorder)
Gamma = buildGamma(y, dy, nvar, polyorder)

# Perform SVD
U, S, V = svd(0*Theta-Gamma, full_matrices=False)
xi0 = V[-1,:]
xi0[np.abs(xi0) < 1e-10] = 0
xi0

output = poolDataLIST(['x', 'y'], xi0, nvar, polyorder)
print(output)

['x']	0.0
['y']	0.0
['x', 'x']	-0.6666666666666667
['x', 'y']	0.0
['y', 'y']	0.6666666666666669
['x', 'x', 'x']	0.0
['x', 'x', 'y']	0.0
['x', 'y', 'y']	0.0
['y', 'y', 'y']	0.0
['x', 'x', 'x', 'x']	0.33333333333333337
['x', 'x', 'x', 'y']	0.0
['x', 'x', 'y', 'y']	0.0
['x', 'y', 'y', 'y']	0.0
['y', 'y', 'y', 'y']	0.0
[['x'], ['y'], ['x', 'x'], ['x', 'y'], ['y', 'y'], ['x', 'x', 'x'], ['x', 'x', 'y'], ['x', 'y', 'y'], ['y', 'y', 'y'], ['x', 'x', 'x', 'x'], ['x', 'x', 'x', 'y'], ['x', 'x', 'y', 'y'], ['x', 'y', 'y', 'y'], ['y', 'y', 'y', 'y']]


In [ ]:
def lorenz_deriv(x_y_z, t0, sigma = 10, beta = 8/3 , rho = 28):
    x, y, z = x_y_z
    return [sigma * (y - x), x * (rho - z) - y, x * y - beta * z]


def solve_lorenz(ic,t ,sigma = 10, beta = 8/3 , rho = 28):
    sol = odeint(lorenz_deriv, ic, t,args=(sigma, beta ,rho))
    return sol

t = np.linspace(0, 10, 10000)
x0 = [0,1,0]
y=solve_lorenz(x0,t)
dy = np.array(lorenz_deriv(y.T,t)).T



# Construct libraries
polyorder = 3
nvar = 3
Theta = buildTheta(y, nvar, polyorder)
Gamma = buildGamma(y, dy, nvar, polyorder)

# Perform SVD
U, S, V = svd(0*Theta-Gamma, full_matrices=False)
xi0 = V[-1,:]
xi0[np.abs(xi0) < 1e-3] = 0

output = poolDataLIST(['x', 'y','z'], xi0, nvar, polyorder)
print(output)

['x']	-0.02619990612069032
['y']	-0.01658837900910595
['z']	0.9914563773429903
['x', 'x']	0.07195924600488096
['x', 'y']	0.04423858778827224
['x', 'z']	-0.004398121796604992
['y', 'y']	-0.04972581326705744
['y', 'z']	0.030482913672945346
['z', 'z']	-0.0739785830338777
['x', 'x', 'x']	0.0
['x', 'x', 'y']	0.0
['x', 'x', 'z']	-0.004283252926572509
['x', 'y', 'y']	0.0
['x', 'y', 'z']	0.0
['x', 'z', 'z']	0.0
['y', 'y', 'y']	0.0
['y', 'y', 'z']	0.0014358931389834074
['y', 'z', 'z']	0.0
['z', 'z', 'z']	0.0014466148160194592
[['x'], ['y'], ['z'], ['x', 'x'], ['x', 'y'], ['x', 'z'], ['y', 'y'], ['y', 'z'], ['z', 'z'], ['x', 'x', 'x'], ['x', 'x', 'y'], ['x', 'x', 'z'], ['x', 'y', 'y'], ['x', 'y', 'z'], ['x', 'z', 'z'], ['y', 'y', 'y'], ['y', 'y', 'z'], ['y', 'z', 'z'], ['z', 'z', 'z']]


In [ ]:
print(V[-1,:])

[-2.61999061e-02 -1.65883790e-02  9.91456377e-01  7.19592460e-02
  4.42385878e-02 -4.39812180e-03 -4.97258133e-02  3.04829137e-02
 -7.39785830e-02  2.74569174e-04 -1.24548641e-04 -4.28325293e-03
 -2.91085511e-04  3.94140949e-04 -3.18092213e-05 -4.81123265e-04
  1.43589314e-03 -5.94751035e-04  1.44661482e-03]
